# Datasets Analysis

In [ ]:
%load_ext autoreload
%autoreload 2

from collections import Counter

from project_paths import paths

from lane_detection_medium.datasets.file_datasets import DatasetMode
from lane_detection_medium.utils.fs import read_yolo_labels
from lane_detection_medium.utils.load import get_label_map


In [ ]:
label_map = get_label_map()
label_map

## Data Loading

In [ ]:
data_dpath = paths.yolo_dpath / "data" / "2023_07_10"

for data_mode in DatasetMode:
    mode_dpath = data_dpath / data_mode.name

    img_dpath = mode_dpath / "images"
    lbl_dpath = mode_dpath / "labels"

    images = list(img_dpath.glob("*.PNG"))
    labels = list(lbl_dpath.glob("*.txt"))

    print(f"--- {data_mode.name.upper()} ---")
    print(f"\tThe number of images: {len(images)}")
    print(f"\tThe number of labels: {len(labels)}")
    print(f"\tThe number of background images: {len(images) - len(labels)}")

    lbl_counter = None 
    for lbl_fpath in labels:
        labels_np = read_yolo_labels(lbl_fpath)[:, 0].astype(int)
        if lbl_counter is None: 
            lbl_counter = Counter(labels_np)
            continue
        lbl_counter += Counter(labels_np)
    print(f"\t{lbl_counter}")

